In [2]:
from tools import *
from tqdm import tqdm
from typing import List, Dict
import json
import requests
from tqdm import tqdm
from typing import List, Dict,Tuple
import json
import requests
import tools

In [3]:
stats = tools.clear_all_graph_data()


🗑️  Starting to clear all graph data...
   Deleting all relationships...
   Deleting all nodes...
   Cleaning up GDS graphs...
   - No 'entity' graph to drop
   Dropping all indexes...
   Dropping all constraints...
✅ Graph cleanup completed!
   - Deleted 956 relationships
   - Deleted 134 nodes
   - Remaining nodes: 0


In [4]:
url = "https://www.gutenberg.org/cache/epub/1727/pg1727.txt"
response = requests.get(url)

In [5]:
def chunk_into_books(text: str) -> List[str]:
    return (
        text.split("PREFACE TO FIRST EDITION")[2]
        .split("FOOTNOTES")[0]
        .strip()
        .split("\nBOOK")[1:]
    )

books = chunk_into_books(response.text)

In [6]:
token_count = [num_tokens_from_string(el) for el in books]
print(
    f"""There are {len(token_count)} books with token sizes:
- avg {sum(token_count) / len(token_count)}
- min {min(token_count)}
- max {max(token_count)}
"""
)

There are 24 books with token sizes:
- avg 6515.208333333333
- min 4459
- max 10760



In [7]:
chunked_books = [chunk_text(book, 1000, 40) for book in books]

In [8]:
#이미 정해져 있는 entity
ENTITY_TYPES = "PERSON,ORGANIZATION,LOCATION,GOD,EVENT,CREATURE,WEAPON_OR_TOOL"

def extract_entities(text: str) -> Tuple[List[Dict], List[Dict]]:
    """
    Extract entities and relationships from text using Bedrock
    
    Args:
        text: Input text to analyze
        
    Returns:
        Tuple of (entities, relationships) as lists of dictionaries
    """
    # Construct prompt
    messages = [
        {"role": "user", "content": tools.create_extraction_prompt(ENTITY_TYPES, text)}
    ]
    
    # Make the LLM call using Bedrock (model parameter is ignored)
    output = tools.chat(messages, model="gpt-4o")  # Will use Bedrock instead
    
    # Parse output and return both entities and relationships
    return tools.parse_extraction_output(output)

In [ ]:
final_nodes = [] 
final_relationship = [] 
number_of_books = 1
for book_i, book in enumerate(
    tqdm(chunked_books[:number_of_books], desc="Processing Books")
):
    for chunk_i, chunk in enumerate(tqdm(book, desc=f"Book {book_i}", leave=False)):
        nodes, relationships = extract_entities(chunk)
        final_nodes.extend(nodes)
        final_relationship.extend(relationships)

Book 0:   0%|          | 0/22 [00:00<?, ?it/s]

⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";MUSE;GOD;Divine being invoked to tell the story of the hero's journey)
|
("entity";TROY;LOCATION;Famous town that was sacked by the hero)
|
("entity";HYPERION;GOD;Sun-god whose cattle were eaten by the hero's men, leading to their destruction)
|
("entity";JOVE;GOD;Father deity, also known as Jupiter or Zeus)
|
("entity";ULYSSES;PERSON;Ingenious hero who travelled far and wide after sacking Troy, trying to return home to his wife and country)
|
("entity";CALYPSO;GOD;Goddess who detained Ulysses in a large cave and wanted to marry him)
|
("entity";MINERVA;GOD;Goddess mentioned in the title who visits Ithaca)
|
("entity";ITHACA;LOCATION;Location visited by Minerva, likely Ulysses' homeland)
|
("entity";TELEMACHUS;PERSON;Person mentioned in the title who challenges the suitors)
|
("entity";SACKING OF TROY;EVENT;The destruction of the famous town of Troy by the hero)
|
("relationship";ULYSSES;TROY;Ulysses sac


Book 0:   5%|▍         | 1/22 [00:13<04:44, 13.53s/it]

)⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";ITHACA;LOCATION;The homeland that the hero is destined to return to after years of wandering)
|
("entity";NEPTUNE;GOD;God of the sea who persistently persecuted the hero and prevented him from getting home)
|
("entity";ETHIOPIANS;PERSON;People who live at the world's end in two halves, one looking West and the other East)
|
("entity";JOVE;GOD;Olympian god, sire of gods and men, who speaks first in the council of gods)
|
("entity";AEGISTHUS;PERSON;Man who made love to Agamemnon's wife and killed Agamemnon, later killed by Orestes)
|
("entity";AGAMEMNON;PERSON;King who was killed by Aegisthus after Aegisthus seduced his wife)
|
("entity";ORESTES;PERSON;Agamemnon's son who killed Aegisthus in revenge for his father's murder)
|
("entity";OLYMPUS;LOCATION;The house of Olympian Jove where the gods meet in council)
|
("entity";HECATOMB;EVENT;Sacrifice of sheep and oxen that Neptune went to accept from the Ethi


Book 0:   9%|▉         | 2/22 [00:25<04:15, 12.79s/it]

...

 take place;9)⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";TELEMACHUS;PERSON;Host who welcomes the visitor, leads them inside, and arranges seating away from the suitors to speak privately about his father)
|
("entity";MINERVA;GOD;Disguised visitor who is welcomed into the house and given hospitality by Telemachus)
|
("entity";SPEAR;WEAPON_OR_TOOL;Weapon carried by Minerva that Telemachus takes and places in the spear-stand)
|
("entity";SPEAR-STAND;WEAPON_OR_TOOL;Stand against a strong bearing-post where spears are stored, containing many spears of Telemachus's father)
|
("entity";ULYSSES;PERSON;Unhappy father of Telemachus whose many spears are stored in the spear-stand)
|
("entity";SUITORS;ORGANIZATION;Noisy and insolent group that Telemachus wants to avoid while speaking with his guest)
|
("entity";DAMASK CLOTH;WEAPON_OR_TOOL;Fine cloth thrown under the richly decorated seat for the guest)
|
("entity";FOOTSTOOL;WEAPON_OR_TOOL;Furniture provided 


Book 0:  32%|███▏      | 7/22 [01:34<03:29, 13.98s/it]

;6)⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";MANSERVANT;PERSON;Male servant who brings wine and pours it out for the guests)
|
("entity";SUITORS;ORGANIZATION;Group that comes in and takes places on benches and seats, enjoys feasting and entertainment)
|
("entity";MEN SERVANTS;ORGANIZATION;Male servants who pour water over the suitors' hands)
|
("entity";MAIDS;ORGANIZATION;Female servants who go round with bread-baskets)
|
("entity";PAGES;ORGANIZATION;Young servants who fill the mixing-bowls with wine and water)
|
("entity";MIXING-BOWLS;WEAPON_OR_TOOL;Vessels that pages fill with wine and water for the suitors)
|
("entity";LYRE;WEAPON_OR_TOOL;Musical instrument brought by a servant and played by Phemius)
|
("entity";PHEMIUS;PERSON;Singer who is compelled by the suitors to perform music for them)
|
("entity";TELEMACHUS;PERSON;Host who speaks quietly to Minerva about the situation, expressing concern about the cost of the entertainment)
|
("entity"


Book 0:  36%|███▋      | 8/22 [01:48<03:14, 13.91s/it]

 forced to provide entertainment for the suitors;7)⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";TELEMACHUS;PERSON;Son who speaks of his missing father and questions the visitor about their identity and origins)
|
("entity";ITHACA;LOCATION;Homeland where Telemachus hopes his father will return, and where the visitor has arrived)
|
("entity";ULYSSES;PERSON;Missing father who has fallen on an ill fate, whose return would frighten the suitors)
|
("entity";MINERVA;GOD;Goddess disguised as Mentes who answers Telemachus's questions about her identity)
|
("entity";MENTES;PERSON;Identity assumed by Minerva, claimed to be son of Anchialus and King of the Taphians)
|
("entity";ANCHIALUS;PERSON;Claimed father of Mentes in Minerva's disguise)
|
("entity";TAPHIANS;ORGANIZATION;People ruled by Mentes according to Minerva's disguise)
|
("entity";SHIP;WEAPON_OR_TOOL;Vessel that Minerva claims to have arrived in with her crew)
|
("entity";CREW;ORGANIZATION;Group of 